In [10]:
import os
import sys

In [14]:
import ccxt
import pandas as pd
import time

In [18]:
# Initialize Binance
binance = ccxt.binance({
    'rateLimit': 1200,
    'enableRateLimit': True,
    # 'apiKey':API_KEY,
    # 'secret':SECRET_KEY
})


In [27]:
def fetch_historical_data(symbol, timeframe, since, limit):
    ohlcv = binance.fetch_ohlcv(symbol, timeframe, since, limit)
    data = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    return data

# Parameters
symbol = 'BTC/USDT'
timeframe = '1h'  # 1m, 5m, 15m, 1h, 1d, etc.
since = binance.parse8601('2024-04-01T00:00:00Z')  # Fetch data since January 1, 2023
limit = 1000  # Fetch up to 1000 candles

In [28]:
historical_data = fetch_historical_data(symbol, timeframe, since, limit)


In [32]:
def fetch_historical_data(symbol, timeframe, since, limit=1000):
    all_data = []
    while True:
        data = binance.fetch_ohlcv(symbol, timeframe, since=since, limit=limit)
        if len(data) == 0:
            break
        all_data += data
        since = data[-1][0] + 1  # Move to the next timestamp after the last one fetched
        time.sleep(binance.rateLimit / 1000)  # Respect rate limit
    return all_data

# Parameters
symbol = 'BTC/USDT'
timeframe = '1h'
since = binance.parse8601('2017-07-01T00:00:00Z')  # Binance started in July 2017

# Fetch data
ohlcv_data = fetch_historical_data(symbol, timeframe, since)

# Convert to DataFrame
df = pd.DataFrame(ohlcv_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# Save to CSV
df.to_csv('BTC_USDT_daily.csv', index=False)

# Display the first few rows
print(df.head())

            timestamp     open     high      low    close     volume
0 2017-08-17 04:00:00  4261.48  4313.62  4261.32  4308.83  47.181009
1 2017-08-17 05:00:00  4308.83  4328.69  4291.37  4315.32  23.234916
2 2017-08-17 06:00:00  4330.29  4345.45  4309.37  4324.35   7.229691
3 2017-08-17 07:00:00  4316.62  4349.99  4287.41  4349.99   4.443249
4 2017-08-17 08:00:00  4333.32  4377.85  4333.32  4360.69   0.972807


In [33]:
df

,timestamp,open,high,low,close,volume
0,2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009
1,2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916
2,2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691
3,2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249
4,2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807
...,...,...,...,...,...,...
59536,2024-06-07 04:00:00,70989.69,71274.70,70989.68,71226.52,1041.733480
59537,2024-06-07 05:00:00,71226.52,71449.98,71170.00,71290.60,1688.264650
59538,2024-06-07 06:00:00,71290.60,71353.06,71162.38,71265.70,589.450230
59539,2024-06-07 07:00:00,71265.70,71310.80,71068.16,71110.00,854.079090
